In [1]:
# This chunk of code make it possible to use src functions
import sys
import os
module_path = os.path.abspath(os.path.join('../src'))

if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings('ignore')

from DSProject import DSProject
from preprocess import *

dsp = DSProject()

In [2]:
from DSProject import min_date_train, max_date_train, min_date_test, max_date_test, train_label, test_label

In [3]:
min_date_text = '20160401'
max_date_text = '20161001'

In [5]:
df = dsp.read_table('sor_cc_ar_filtered_' + min_date_text + '_' + max_date_text, 'preprocess', index_col=0)

In [ ]:
df.head()

In [ ]:
df = dsp.read_data('EDW_SOR_CC_AR')

In [6]:
df0 = df

In [7]:
df = df0
df = delete_missing_columns(df0)
df = delete_single_value_columns(df)

Deleted from missings list = ['FSVC_TMT_OUTC_TP_ID', 'RSTC_DFCLT_LVL_ID', 'FSVC_UTLZ_SEG_ID', 'FSVC_UTLZ_RNG_ID', 'PTNL_ESR_RNG_ID', 'PD_ID', 'ALT_CST_EXP_DT', 'CLT_TP_ID', 'CLOSE_DT', 'CRT_DT', 'AFF_MBR_TP_REL_YR', 'AFF_MBR_ORG_GRP_ID', 'CR_LMT_TEMP_EFF_DT', 'CR_LMT_TEMP_END_DT']
Number of column deleted = 14
Delete single value column list
['AR_LVL_IND', 'AR_SCR_TP_ID', 'FSVC_LMT_TP_ID', 'FSVC_RPYMT_TP_ID', 'CST_ORG_NBR', 'ORIG_CCY_ID', 'ACR_MISC_MEMO_AMT', 'BNS_BUCK_BAL', 'BNS_BUCK_USED', 'BNS_BUCK_RNG_ID']
Number of column deleted
10


In [8]:
df['HASH_AR_ID'] = df['HASH_AR_ID'].astype(str)
df['POSN_DT'] = parse_dates(df['POSN_DT'], '%Y-%m-%d')

In [9]:
target = dsp.read_table('group_payment_target', 'feature', use_schema=True)

In [10]:
prep = pd.merge(target[['HASH_AR_ID', 'FileDate', 'Previous_FileDate', 'Next_FileDate']], 
                df, on=['HASH_AR_ID'], how ='inner', suffixes=['','_y'])

In [11]:
df = prep.loc[(prep['POSN_DT'] < prep['FileDate']) & (prep['POSN_DT'] >= prep['Previous_FileDate'])]
print("Merge coverage")
print(len(df.index)/len(target.index))

Merge coverage
0.36107692978201944


In [12]:
dsp.write_table(df, 'sor_cc_ar_' + min_date_text + '_' + max_date_text, 'feature')

In [13]:
print(len(df.index))

389385


In [14]:
dsp.data_directory

'/Users/jah/jah/work/consulting/kbank/code/data/201510_201610'

In [15]:
df_train, df_test = filter_dates(df, min_date_train, max_date_train, min_date_test, max_date_test)

In [16]:
len(df_train.index)

199082

In [17]:
len(df_test.index)

62276

In [18]:
dsp.write_table(pd.concat([df_train, df_test]), 'sor_cc_ar_' + min_date_text + '_' + max_date_text, 'feature')